In [1]:
import datetime
import random

import numpy as np
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm

from common import *

Using TensorFlow backend.


In [2]:
print(".. setting base configuration & params ..")

initialize()

best_model_filepath = './saved-models/transfer_learning.hdf5'
training_epochs = 50
batch_size = 20
images_size = 224

.. setting base configuration & params ..


In [3]:
(train_tensors, X_train, y_train,
test_tensors, X_test, y_test, 
valid_tensors, X_validate, y_validate, 
duration_loading) = load_and_split_data(images_size=images_size)

  2%|▏         | 31/1408 [00:00<00:04, 306.37it/s]

.. loading & splitting data ..


100%|██████████| 352/352 [00:01<00:00, 314.97it/s]


In [4]:
# external_model = InceptionV3(weights='imagenet', include_top=False)
# external_model = ResNet50(weights='imagenet', include_top=False)
external_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [5]:
print(".. constructing the transfer model ..")

for layer in external_model.layers:
    layer.trainable = False

temp = Conv2D(filters=128, kernel_size=2, padding='same')(external_model.layers[-13].output)

temp = BatchNormalization()(temp)
temp = Activation('relu')(temp)
temp = MaxPooling2D(pool_size=(2, 2))(temp)
temp = GlobalAveragePooling2D()(temp)
predictions = Dense(2, activation='softmax')(temp)

model = Model(input=external_model.input, output=predictions)

model.summary()    

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    

.. constructing the transfer model ..
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [6]:
duration_training = train_single_model(model, 
                                       best_model_filepath, 
                                       train_tensors, y_train, 
                                       valid_tensors, y_validate, 
                                       training_epochs, 
                                       batch_size)


.. training the model ..
Train on 1408 samples, validate on 352 samples
Epoch 1/50
1408/1408 [==============================] - 673s 478ms/step - loss: 0.3401 - acc: 0.8409 - val_loss: 0.3514 - val_acc: 0.8778

Epoch 00001: val_loss improved from inf to 0.35137, saving model to ./saved-models/transfer_learning.hdf5
Epoch 2/50
1408/1408 [==============================] - 677s 481ms/step - loss: 0.2705 - acc: 0.8665 - val_loss: 0.2982 - val_acc: 0.8977

Epoch 00002: val_loss improved from 0.35137 to 0.29820, saving model to ./saved-models/transfer_learning.hdf5
Epoch 3/50
1408/1408 [==============================] - 681s 484ms/step - loss: 0.2552 - acc: 0.8786 - val_loss: 0.1771 - val_acc: 0.9233

Epoch 00003: val_loss improved from 0.29820 to 0.17715, saving model to ./saved-models/transfer_learning.hdf5
Epoch 4/50
1408/1408 [==============================] - 683s 485ms/step - loss: 0.2418 - acc: 0.8899 - val_loss: 0.3148 - val_acc: 0.8864

Epoch 00004: val_loss did not improve from 0.1

1408/1408 [==============================] - 721s 512ms/step - loss: 0.1687 - acc: 0.9219 - val_loss: 0.7838 - val_acc: 0.8125

Epoch 00041: val_loss did not improve from 0.14332
Epoch 42/50
1408/1408 [==============================] - 722s 513ms/step - loss: 0.1674 - acc: 0.9261 - val_loss: 0.2506 - val_acc: 0.8665

Epoch 00042: val_loss did not improve from 0.14332
Epoch 43/50
1408/1408 [==============================] - 699s 497ms/step - loss: 0.1652 - acc: 0.9254 - val_loss: 0.2539 - val_acc: 0.8608

Epoch 00043: val_loss did not improve from 0.14332
Epoch 44/50
1408/1408 [==============================] - 719s 510ms/step - loss: 0.1715 - acc: 0.9183 - val_loss: 0.4667 - val_acc: 0.8182

Epoch 00044: val_loss did not improve from 0.14332
Epoch 45/50
1408/1408 [==============================] - 700s 497ms/step - loss: 0.1736 - acc: 0.9226 - val_loss: 0.2341 - val_acc: 0.8608

Epoch 00045: val_loss did not improve from 0.14332
Epoch 46/50
1408/1408 [==============================] - 

In [7]:
print(".. loading best weights ..")

model.load_weights(best_model_filepath)

.. loading best weights ..


In [8]:
print(".. testing the model on Test data ..")
section_start_time = datetime.datetime.utcnow()

test_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(test_tensors)]

test_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/440 [00:00<?, ?it/s]

.. testing the model on Test data ..


100%|██████████| 440/440 [03:08<00:00,  2.32it/s]


In [9]:
print(".. testing the model on Training data ..")
section_start_time = datetime.datetime.utcnow()

train_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(train_tensors)]

train_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/1408 [00:00<?, ?it/s]

.. testing the model on Training data ..


100%|██████████| 1408/1408 [09:06<00:00,  2.68it/s]


In [10]:
print(".. testing the model on Validation data ..")
section_start_time = datetime.datetime.utcnow()

validation_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(valid_tensors)]

validation_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/352 [00:00<?, ?it/s]

.. testing the model on Validation data ..


100%|██████████| 352/352 [02:19<00:00,  2.83it/s]


In [11]:
print("________________________________________________________")
print("TIMING SUMMARY:\n")
print("loading duration: {0:.1f} seconds".format(duration_loading))
print("training duration: {0:.1f} seconds".format(duration_training))

print("________________________________________________________")
print("MODEL PERFORMANCE ON TEST DATA:\n")
print("predict duration: {0:.1f} seconds".format(test_data_predictions_time))
false_positive_images, false_negative_images = summarize_model_performance(X_test, y_test, test_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON TRAIN DATA:\n")
print("predict duration: {0:.1f} seconds".format(train_data_predictions_time))
_, _ = summarize_model_performance(X_train, y_train, train_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON VALIDATION DATA:\n")
print("predict duration: {0:.1f} seconds".format(validation_data_predictions_time))
_, _ = summarize_model_performance(X_validate, y_validate, validation_data_predictions)

________________________________________________________
TIMING SUMMARY:

loading duration: 8.5 seconds
training duration: 43418.8 seconds
________________________________________________________
MODEL PERFORMANCE ON TEST DATA:

predict duration: 188.8 seconds
all:  440
all_positives:  81
all_negatives:  359
true_positives:  79
true_negatives:  340
false_positives:  19
false_negatives:  2
RECALL: 0.98
SPECIFICITY: 0.95
ACCURACY: 0.95
PRECISION: 0.81
F1 SCORE: 0.88
FP RATE / ERROR I: 0.05
FN RATE / ERROR II: 0.02
________________________________________________________
MODEL PERFORMANCE ON TRAIN DATA:

predict duration: 546.3 seconds
all:  1408
all_positives:  316
all_negatives:  1092
true_positives:  298
true_negatives:  1018
false_positives:  74
false_negatives:  18
RECALL: 0.94
SPECIFICITY: 0.93
ACCURACY: 0.93
PRECISION: 0.80
F1 SCORE: 0.87
FP RATE / ERROR I: 0.07
FN RATE / ERROR II: 0.06
________________________________________________________
MODEL PERFORMANCE ON VALIDATION DATA:



## False Positives (first 20):

In [12]:
display_images(false_positive_images[:20])

## False Negatives (first 20):

In [13]:
display_images(false_negative_images[:20])